insert data
* insert
  * create
* insert_one
* insert_many
* save

get data
* get / find_one
* find
  * to_list
  * first_or_none
* find_many
* find_all

sync


when we would like to create application 

we want to create for users

that why our first class will be user

pydantic + beanie = ❤️


In [155]:
from pydantic import BaseModel


class User(BaseModel):
    name: str
    surname: str
    email: str

type(User)

pydantic._internal._model_construction.ModelMetaclass

In [156]:
not_hot_adam = User(name="Adam", surname="Brzyzek", email="nothotadam@gmail.com")
not_hot_adam

User(name='Adam', surname='Brzyzek', email='nothotadam@gmail.com')

In [157]:
not_hot_adam.model_dump_json()

'{"name":"Adam","surname":"Brzyzek","email":"nothotadam@gmail.com"}'

In [158]:
not_hot_adam.model_dump()

{'name': 'Adam', 'surname': 'Brzyzek', 'email': 'nothotadam@gmail.com'}

In [159]:
from beanie import Document


class User(Document):
    name: str
    surname: str
    email: str

type(User)

pydantic._internal._model_construction.ModelMetaclass

if you run it you will get 'CollectionWasNotInitialized'

first initialize Collection

In [160]:
import os

from beanie import init_beanie
from motor.motor_asyncio import AsyncIOMotorClient


client = AsyncIOMotorClient(os.getenv("MONGODB_URI"))
await init_beanie(
    database=client.workshop,
    document_models=[User],
    multiprocessing_mode=True,
)
client.drop_database(name_or_database=client.workshop)


<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/coolig/.pyenv/versions/3.11.9/lib/python3.11/asyncio/futures.py:387]>

In [161]:
hot_adam = User(name="Adam", surname="Brzyzek", email="hotadam@gmail.com")
hot_adam


User(id=None, revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')

In [164]:
hot_adam.model_dump()

{'id': None,
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [163]:
User.insert(hot_adam)

<coroutine object Document.insert at 0x10dca3450>

coroutine

In [165]:
hot_adam.model_dump()

{'id': None,
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [166]:
await User.insert(hot_adam)

User(id=ObjectId('66c3ab63d4bd4b7536884dee'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')

In [ ]:
# id i revision_id

In [167]:
hot_adam.model_dump()


{'id': '66c3ab63d4bd4b7536884dee',
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [168]:
client.drop_database(name_or_database=client.workshop)


<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/coolig/.pyenv/versions/3.11.9/lib/python3.11/asyncio/futures.py:387]>

In [169]:
hot_adam = User(name="Adam", surname="Brzyzek", email="hotadam@gmail.com")
hot_adam
await User.insert_one(hot_adam)


User(id=ObjectId('66c3ab9dd4bd4b7536884def'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')

In [170]:
hot_adam.model_dump()

{'id': '66c3ab9dd4bd4b7536884def',
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [171]:
hot_adam = User(name="Adam", surname="Brzyzek", email="hotadam@gmail.com")
await User.create(hot_adam)
hot_adam.model_dump()

{'id': '66c3abadd4bd4b7536884df0',
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [172]:
adam = await User.get(hot_adam.id)
adam, type(adam)

(User(id=ObjectId('66c3abadd4bd4b7536884df0'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 __main__.User)

diffrence between get and find one

In [173]:
await User.find_one(User.id == hot_adam.id)

User(id=ObjectId('66c3abadd4bd4b7536884df0'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')

find more powerfull

In [175]:
result = User.find()
print(result, type(result))

<coroutine object BaseCursorQuery.to_list at 0x10dfde840> <class 'coroutine'>


In [179]:
result = User.find().to_list()
# print(result)
# print(len(result))


/var/folders/8t/rr7grrqn4vg0mt1wmd21h3w80000gn/T/ipykernel_38731/990964848.py:1: RuntimeWarning: coroutine 'BaseCursorQuery.to_list' was never awaited
  result = User.find().to_list()


In [180]:
await result

[User(id=ObjectId('66c3ab9dd4bd4b7536884def'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c3abadd4bd4b7536884df0'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')]

In [181]:
result = await User.find().first_or_none()
print(result)
print(type(result))

id=ObjectId('66c3ab9dd4bd4b7536884def') revision_id=None name='Adam' surname='Brzyzek' email='hotadam@gmail.com'
<class '__main__.User'>


pymongo syntax - ble string XD

In [182]:
await User.find({"_id": hot_adam.id}).to_list()


[User(id=ObjectId('66c3abadd4bd4b7536884df0'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')]

In [183]:
await User.find_many().to_list()


[User(id=ObjectId('66c3ab9dd4bd4b7536884def'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c3abadd4bd4b7536884df0'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')]

In [184]:
await User.find_all().to_list()


[User(id=ObjectId('66c3ab9dd4bd4b7536884def'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c3abadd4bd4b7536884df0'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')]